# Мультиклассовая классификация

В данном ноутбуке рассмотрим подход к задаче с точки зрения мультиклассовой классификации. То есть, мы не будем учитывать порядок классов, а попробуем просто спрогнозировать их, как равнозначные.

В ноутбуке с анализом данных, исходя из корреляций признаков объектов и целевого значения, удалось установить, что многие признаки имеют высокую корреляцию с таргетом. Таким образом, стоит попробовать модель LogisticRegression.


In [4]:
import pandas as pd
import numpy as np

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/intern_task_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/intern_task_test.csv")

Предварительно создадим функцию для оценки требуемой метрики ранжирования NDCG_5.

Из анализа датасета, удалось установить, что количество документов, соответсвующих поисковой сессии, не имет фиксированную длину. А также может быть меньше 5.

In [1]:
from sklearn.metrics import ndcg_score
from sklearn.metrics import accuracy_score


def NDCG_atK_score(test_Data, pred_Data, k, logits=False):
  query_ids = test_Data['query_id'].unique()
  y_true_sessions = [[rank for rank in test_Data[test_Data['query_id'] == id]['rank']] for id in query_ids]
  y_pred_sessions = [[rank for rank in pred_Data[pred_Data['query_id'] == id]['rank']] for id in query_ids]
  if logits:
    print(y_true_sessions)
    print(y_pred_sessions)
  ndcg_scores = [ndcg_score([y_true_sessions[i]], [y_pred_sessions[i]], k=k) for i in range(len(y_true_sessions)) if len(y_true_sessions[i]) > 1]
  return np.mean(ndcg_scores)

Подготовим данные: оставим из сильно коррелирующих признаков объектов только один признак (такие признаки были установлены при анализе датасета). Применим стандартизацию и используем OHE для кодирования категориальных признаков.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

to_drop = ['feature_14', 'feature_17', 'feature_19', 'feature_20',
       'feature_29', 'feature_34', 'feature_35', 'feature_44',
       'feature_74', 'feature_79', 'feature_84', 'feature_89',
       'feature_94', 'feature_114', 'feature_119', 'feature_120',
       'feature_121', 'feature_122', 'feature_123', 'feature_124',
       'feature_142', 'feature_143']

cat_features = ['feature_95', 'feature_96', 'feature_97', 'feature_98', 'feature_99']

df_train = pd.get_dummies(df_train, columns=cat_features)
df_test = pd.get_dummies(df_test, columns=cat_features)

X_train = df_train.drop(columns=to_drop)
X_train = X_train.drop(columns=['rank', 'query_id'])
X_test = df_test.drop(columns=to_drop)
X_test = X_test.drop(columns=['rank', 'query_id'])
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg = LogisticRegression(max_iter=1500, class_weight='balanced')
logreg.fit(X_train, df_train['rank'])

LogisticRegression(class_weight='balanced', max_iter=1500)

In [ ]:
outputs = logreg.predict(X_test)
df_tmp = df_test.copy()
df_tmp['rank'] = outputs
print("NDCG@5 score : ", NDCG_atK_score(df_test, df_tmp, k=5))

NDCG@5 score :  0.2890831176622812


Так как NDCG@k принимает значения из диапазона $[0, 1]$, при этом лучше, чем ближе к 1, то можно сказать, что модель справилась плохо. Нужно искать другой подход к решению задачи.

### Catboost Classifier

Попробуем теперь воспользоваться градиентным бустингом с помощью фреймворка CatBoost. Вновь будем решать задачу с точки зрения мультиклассовой классификации.

In [2]:
!pip install catboost

In [11]:
df_train = pd.read_csv("/content/drive/MyDrive/intern_task_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/intern_task_test.csv")

In [12]:
cat_features = ['feature_95', 'feature_96', 'feature_97', 'feature_98', 'feature_99', 'feature_1', 'feature_3', 'feature_28']
for feature in cat_features:
  df_train[feature] = df_train[feature].astype(int)
  df_test[feature] = df_test[feature].astype(int)

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(cat_features=cat_features, task_type="GPU")

model.fit(df_train.drop(columns=['query_id', 'rank']), df_train['rank'])

In [14]:
outputs = model.predict(df_test.drop(columns=['query_id', 'rank']))
df_tmp = df_test.copy()
df_tmp['rank'] = outputs
print("NDCG@5 score : ", NDCG_atK_score(df_test, df_tmp, k=5))

NDCG@5 score :  0.44620098274901776


С помощью градиентого бустинга получилось существенно улучшить показатели модели по метрике. Однако, результаты все еще нельзя назвать хорошими.

Будем искать совершенно другой подход.